#### Set-ups and Imports

In [11]:
import pickle
import re
import os

import random
import numpy as np
import torch
from random import shuffle
import argparse
import pickle

import collections

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import sys
sys.path.append("..")

from model.BiLSTM import *
from model.BERT import *

from torch.utils.data import DataLoader, TensorDataset
from torch.utils.data.distributed import DistributedSampler
from torch.utils.data.sampler import RandomSampler, SequentialSampler
from tqdm import tqdm, trange

from util.optimization import BERTAdam
from util.processor import *

from util.tokenization import *

from util.evaluation import *

import logging
logging.basicConfig(format = '%(asctime)s - %(levelname)s - %(name)s -   %(message)s', 
                    datefmt = '%m/%d/%Y %H:%M:%S',
                    level = logging.INFO)
logger = logging.getLogger(__name__)

from sklearn.metrics import classification_report

# this imports most of the helpers needed to eval the model
from run_classifier_sa import *

In [24]:
# Note that this notebook only supports single GPU evaluation
# which is sufficient for most of tasks by using lower batch size.
IS_CUDA = False
if IS_CUDA:
    CUDA_DEVICE = "cuda:5"
    device = torch.device(CUDA_DEVICE)
    n_gpu = torch.cuda.device_count()
    logger.info("device %s in total n_gpu %d distributed training", device, n_gpu)
else:
    # bad luck, we are on CPU now!
    logger.info("gpu is out of the picture, let us use CPU")
    device = torch.device("cpu")

10/28/2020 20:13:57 - INFO - run_classifier_sa -   gpu is out of the picture, let us use CPU


#### Tasks specifications

In [30]:
TASK_NAME = "SST3"   # this is the 3-class classification on SST
                     # where the classes are 
                     # " negative", "positive" and "neutural".
                     # [0,0.2] && [0.4,0.6] && [0.8,1.0]
# "../../data/uncased_L-12_H-768_A-12/" is for the default BERT-base pretrain
MODEL_PATH = "../../results/SST3/checkpoint.bin"
DATA_DIR = "../../data/dataset/SST/"
EVAL_BATCH_SIZE = 24 # you can tune this down depends on GPU you have.

# This loads the task processor for you.
processors = {
    "IMDb":IMDb_Processor,
    "SemEval":SemEval_Processor,
    "SST5":SST5_Processor,
    "SST2":SST2_Processor,
    "SST3":SST3_Processor,
    "Yelp5":Yelp5_Processor,
    "Yelp2":Yelp2_Processor,
    "AdvSA":AdvSA_Processor
}

processor = processors[TASK_NAME]()
label_list = processor.get_labels()

In [34]:
state_dict = torch.load(MODEL_PATH, map_location='cpu')
state_dict

OrderedDict([('bert.embeddings.word_embeddings.weight',
              tensor([[-0.0314, -0.0045,  0.0182,  ..., -0.0309,  0.0204, -0.0345],
                      [-0.0295, -0.0486,  0.0746,  ..., -0.0363,  0.0262, -0.0108],
                      [-0.0328, -0.0582, -0.0149,  ..., -0.0931,  0.0444,  0.0221],
                      ...,
                      [-0.0337, -0.0518, -0.0280,  ..., -0.0174,  0.0078, -0.0010],
                      [-0.0022, -0.0297, -0.0167,  ..., -0.0472, -0.0006,  0.0128],
                      [-0.0631, -0.0144, -0.0232,  ...,  0.0072, -0.0704, -0.0479]])),
             ('bert.embeddings.position_embeddings.weight',
              tensor([[-0.0109, -0.0074,  0.0046,  ..., -0.0242,  0.0088,  0.0046],
                      [ 0.0007,  0.0051,  0.0089,  ..., -0.0015,  0.0143,  0.0176],
                      [-0.0104, -0.0057,  0.0210,  ...,  0.0021,  0.0210,  0.0118],
                      ...,
                      [-0.0242,  0.0173, -0.0074,  ..., -0.0057, -0.009

#### Get specific models, optimizer (not needed), and tokenizer

In [31]:
model, optimizer, tokenizer = \
    getModelOptimizerTokenizer(model_type="BERTPretrain",
                               vocab_file=MODEL_DIR + "vocab.txt",
                               embed_file=None,
                               bert_config_file=MODEL_DIR + "bert_config.json",
                               init_checkpoint=MODEL_PATH,
                               label_list=label_list,
                               do_lower_case=True,
                               # below is not required for eval
                               num_train_steps=20,
                               learning_rate=2e-5,
                               base_learning_rate=2e-5,
                               warmup_proportion=0.1)
model = model.to(device) # send the model to device

10/28/2020 20:20:41 - INFO - run_classifier_sa -   model = BERTPretrain


init_weight = True


RuntimeError: Error(s) in loading state_dict for BertForSequenceClassification:
	Missing key(s) in state_dict: "bert.embeddings.word_embeddings.weight", "bert.embeddings.position_embeddings.weight", "bert.embeddings.token_type_embeddings.weight", "bert.embeddings.LayerNorm.gamma", "bert.embeddings.LayerNorm.beta", "bert.encoder.layer.0.attention.self.query.weight", "bert.encoder.layer.0.attention.self.query.bias", "bert.encoder.layer.0.attention.self.key.weight", "bert.encoder.layer.0.attention.self.key.bias", "bert.encoder.layer.0.attention.self.value.weight", "bert.encoder.layer.0.attention.self.value.bias", "bert.encoder.layer.0.attention.output.dense.weight", "bert.encoder.layer.0.attention.output.dense.bias", "bert.encoder.layer.0.attention.output.LayerNorm.gamma", "bert.encoder.layer.0.attention.output.LayerNorm.beta", "bert.encoder.layer.0.intermediate.dense.weight", "bert.encoder.layer.0.intermediate.dense.bias", "bert.encoder.layer.0.output.dense.weight", "bert.encoder.layer.0

#### Load datasets and features

In [28]:
test_examples = processor.get_test_examples(DATA_DIR)
test_features = \
    convert_examples_to_features(
        test_examples,
        label_list,
        512,
        tokenizer)

all_input_ids = torch.tensor([f.input_ids for f in test_features], dtype=torch.long)
all_input_mask = torch.tensor([f.input_mask for f in test_features], dtype=torch.long)
all_segment_ids = torch.tensor([f.segment_ids for f in test_features], dtype=torch.long)
all_label_ids = torch.tensor([f.label_id for f in test_features], dtype=torch.long)
all_seq_len = torch.tensor([[f.seq_len] for f in test_features], dtype=torch.long)

test_data = TensorDataset(all_input_ids, all_input_mask, all_segment_ids,
                          all_label_ids, all_seq_len)
test_dataloader = DataLoader(test_data, batch_size=EVAL_BATCH_SIZE, shuffle=False)

100%|██████████| 1063/1063 [00:00<00:00, 2755.95it/s]


####  Actual evaluation loop

In [29]:
# we did not exclude gradients, for attribution methods
model.eval() # this line will deactivate dropouts
test_loss, test_accuracy = 0, 0
nb_test_steps, nb_test_examples = 0, 0
# we don't need gradient in this case.
for step, batch in enumerate(tqdm(test_dataloader, desc="Iteration")):
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    input_ids, input_mask, segment_ids, label_ids, seq_lens = batch
    # truncate to save space and computing resource
    max_seq_lens = max(seq_lens)[0]
    input_ids = input_ids[:,:max_seq_lens]
    input_mask = input_mask[:,:max_seq_lens]
    segment_ids = segment_ids[:,:max_seq_lens]

    input_ids = input_ids.to(device)
    input_mask = input_mask.to(device)
    segment_ids = segment_ids.to(device)
    label_ids = label_ids.to(device)
    seq_lens = seq_lens.to(device)

    # intentially with gradient
    tmp_test_loss, logits = \
        model(input_ids, segment_ids, input_mask, seq_lens,
                device=device, labels=label_ids)

    logits = F.softmax(logits, dim=-1)
    logits = logits.detach().cpu().numpy()
    label_ids = label_ids.to('cpu').numpy()
    outputs = np.argmax(logits, axis=1)
    tmp_test_accuracy=np.sum(outputs == label_ids)

    test_loss += tmp_test_loss.mean().item()
    test_accuracy += tmp_test_accuracy

    nb_test_examples += input_ids.size(0)
    nb_test_steps += 1
    
test_loss = test_loss / nb_test_steps
test_accuracy = test_accuracy / nb_test_examples

result = collections.OrderedDict()
result = {'test_loss': test_loss,
            'test_accuracy': test_accuracy}
logger.info("***** Eval results *****")
for key in result.keys():
    logger.info("  %s = %s\n", key, str(result[key]))

Iteration: 100%|██████████| 45/45 [00:23<00:00,  1.90it/s]
10/28/2020 20:18:04 - INFO - run_classifier_sa -   ***** Eval results *****
10/28/2020 20:18:04 - INFO - run_classifier_sa -     test_loss = 1.1122603098551431

10/28/2020 20:18:04 - INFO - run_classifier_sa -     test_accuracy = 0.2737535277516463

